In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install --upgrade gensim
!pip install optuna imblearn sentence-transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 39.1 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')
from imblearn.over_sampling import SMOTE

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

from sentence_transformers import SentenceTransformer

import optuna
from optuna.samplers import TPESampler

import pickle

import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset

from typing import Tuple

import random

In [ ]:
def set_seed(seed: int = 42) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ["PYTHONHASHSEED"] = str(seed)

In [ ]:
path_ukara = "/content/drive/My Drive/Research/AES Backtranslate"

# Raw dataset
path_train_A = path_ukara + "/Raw/Data A/train_A.csv"
path_val_A = path_ukara + "/Raw/Data A/val_A.csv"
path_test_A = path_ukara + "/Raw/Data A/test_A.csv"
path_train_B = path_ukara + "/Raw/Data B/train_B.csv"
path_val_B = path_ukara + "/Raw/Data B/val_B.csv"
path_test_B = path_ukara + "/Raw/Data B/test_B.csv"

# Preprocess and back translate
path_train_A_p3 = path_ukara + "/Preprocess3/Data A/train_A.csv"
path_val_A_p3 = path_ukara + "/Preprocess3/Data A/val_A.csv"
path_test_A_p3 = path_ukara + "/Preprocess3/Data A/test_A.csv"
path_train_B_p3 = path_ukara + "/Preprocess3/Data B/train_B.csv"
path_val_B_p3 = path_ukara + "/Preprocess3/Data B/val_B.csv"
path_test_B_p3 = path_ukara + "/Preprocess3/Data B/test_B.csv"

# Preprocess
path_train_A_p_only = path_ukara + "/Preprocess-Raw/Data A/train_A.csv"
path_val_A_p_only = path_ukara + "/Preprocess-Raw/Data A/val_A.csv"
path_test_A_p_only = path_ukara + "/Preprocess-Raw/Data A/test_A.csv"
path_train_B_p_only = path_ukara + "/Preprocess-Raw/Data B/train_B.csv"
path_val_B_p_only = path_ukara + "/Preprocess-Raw/Data B/val_B.csv"
path_test_B_p_only = path_ukara + "/Preprocess-Raw/Data B/test_B.csv"

# Backgranslate
path_train_A_bt = path_ukara + '/Backtranslate/Data A/train_A.csv'
path_train_B_bt = path_ukara + '/Backtranslate/Data B/train_B.csv'

path_spellcheck = path_ukara + '/spellcheck.txt'

In [ ]:
train_map = {
  "A": path_train_A,
  "B": path_train_B,
  "A-bt": path_train_A_bt,
  "B-bt": path_train_B_bt,
  "B-p3": path_train_B_p_only,
  "A-p3": path_train_A_p_only,
  "B-p3-bt": path_train_B_p3,
  "A-p3-bt": path_train_A_p3,
}

valid_map = {
  "A": path_val_A,
  "B": path_val_B,
  "A-bt": path_val_A,
  "B-bt": path_val_B,
  "A-p3": path_val_A_p_only,
  "B-p3": path_val_B_p_only,
  "A-p3-bt": path_val_A_p3,
  "B-p3-bt": path_val_B_p3,
}

test_map = {
  "A": path_test_A,
  "B": path_test_B,
  "A-bt": path_test_A,
  "B-bt": path_test_B,
  "A-p3": path_test_A_p_only,
  "B-p3": path_test_B_p_only,
  "A-p3-bt": path_test_A_p3,
  "B-p3-bt": path_test_B_p3,
}

dictSoal = {}
for key, value in train_map.items():
  dictSoal[key] = pd.read_csv(value)

dictDev = {}
for key, value in valid_map.items():
  dictDev[key] = pd.read_csv(value)

dictTest = {}
for key, value in test_map.items():
  dictTest[key] = pd.read_csv(value)

In [ ]:
# Drop rows with empty responses
for key, value in dictSoal.items():
  dictSoal[key] = dictSoal[key].dropna(subset=["RESPONSE"], axis=0)

In [ ]:
dictTest["B"].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 974 entries, 0 to 973
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   RES_ID    974 non-null    object
 1   RESPONSE  974 non-null    object
 2   LABEL     974 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 23.0+ KB


In [ ]:
print("TRAIN")
print(dictSoal["A"].LABEL.value_counts()) # 40.31%
print(dictSoal["B"].LABEL.value_counts()) # 81.54%
print(77/191, 95/191, 95-77)
print(137/168, 151/168, 151-137)

TRAIN
LABEL
1    191
0     77
Name: count, dtype: int64
LABEL
1    168
0    137
Name: count, dtype: int64
0.4031413612565445 0.4973821989528796 18
0.8154761904761905 0.8988095238095238 14


# Preprocessing

### Abbreviation Normalization

In [ ]:
df = pd.read_csv(path_ukara + '/dictNorm.csv', index_col=0)
df = df.where(pd.notnull(df), ' ')
dictNorm = df.to_dict()["after"]

def norm_text(text, printer=False):
  if printer:
    print(dictNorm)
  for wBefore, wAfter in dictNorm.items():
      text = wAfter if text == wBefore else text
  text = text.replace('\n', ' ').replace('\\', ' ')
  return text

### Misspelling correction

In [ ]:
def vectorize(dfofSentences, embeddingModel: SentenceTransformer):
  embeddings = embeddingModel.encode(dfofSentences.to_numpy())
  return embeddings

def findLen(dfofSentences):
  res = []
  for sent in dfofSentences:
    sent = str(sent)
    if len(sent)<0:
      res.append(0)
    else:
      res.append(len(sent.split()))
  resnew = np.asarray(res, dtype="float32")
  datalen_mean = np.mean(resnew)
  datalen_std = np.std(resnew)
  resnew -= datalen_mean
  return resnew / datalen_std

## Embedding Model

In [ ]:
embeddingModel = SentenceTransformer('firqaaa/indo-sentence-bert-base')
embeddingModel2 = SentenceTransformer('sentence-transformers/paraphrase-xlm-r-multilingual-v1')

# Dataset

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

class AesDataset(Dataset):
  def __init__(self, x, y):
      self.x = torch.tensor(x, dtype=torch.float32).to(device)
      self.y = torch.tensor(y, dtype=torch.float32).to(device)

  def __len__(self):
    return len(self.x)

  def __getitem__(self, idx) -> Tuple[torch.Tensor, torch.Tensor]:
    return self.x[idx], self.y[idx]

Using cpu device


# Model

In [ ]:
class ModelParams():
  def __init__(self, epochs = 8, init_bs = 12, bs_inc=4, inc_freq=3, smote=True, model="nn1", embedding="indo"):
    self.epochs = epochs
    self.init_bs = init_bs
    self.bs_inc = bs_inc
    self.inc_freq = inc_freq
    self.smote = smote
    self.model = model
    self.embedding = embedding


In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(769, 64),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits


In [ ]:
class NeuralNetwork2(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(769, 64),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits


# Train

In [ ]:
import math

def get_loader(dataset: AesDataset, bs):
  return DataLoader(dataset, batch_size=bs)

def get_batchsize(params: ModelParams, epoch, max_epoch):
  bs = params.init_bs + (math.floor(epoch/(max_epoch/(params.inc_freq+1))) *  params.bs_inc)
  return bs

def train(dataset: AesDataset, model: nn.Module, params: ModelParams):
  loss_function = nn.BCELoss()
  optimizer = optim.AdamW(params=model.parameters(), lr=0.001, weight_decay=1e-3)
  for epoch in range(params.epochs):
    curr_bs = get_batchsize(params, epoch, params.epochs)
    dataloader = get_loader(dataset, curr_bs)
    for data in dataloader:
      inputs, labels = data

      optimizer.zero_grad()

      outputs = model(inputs)

      loss = loss_function(outputs.reshape(-1), labels)
      loss.backward()

      optimizer.step()

  return model

# Run

In [ ]:
def evaluate_model(model, X_ans, Y_ans, dataloader: DataLoader, verbose=False, debug=False):
  Y_pred = []
  incorrect_preds = {
      'label': [],
      'pred': [],
      'ques': [],
  }

  model.eval()
  i = 0
  with torch.no_grad():
    for inputs, labels in dataloader:
      outputs = model(inputs).squeeze()
      predictions = (outputs > 0.5).long()
      if (debug and predictions.item() != int(labels.item())):
        incorrect_preds['pred'].append(predictions.item())
        incorrect_preds['label'].append(int(labels.item()))
        incorrect_preds['ques'].append(X_ans[i])

        if (verbose):
          print(predictions.item(), int(labels.item()))
          print(X_ans[i])
      i = i + 1

      Y_pred.extend(predictions.reshape(-1).tolist())
  f1 = f1_score(Y_ans, Y_pred)



  if (verbose):
    print("Confusion Matrix prediction: TN(00), FP(01), FN(10), TP(11) \n",
          confusion_matrix(Y_ans, Y_pred).ravel()) #tn, fp, fn, tp

  return f1, incorrect_preds


##### Run without cv

In [ ]:
def run(params: ModelParams,  problems=["A"], fixed_seed=True, verbose=False):
    avg_f1_test = 0
    avg_f1_val = 0

    if(fixed_seed):
      set_seed(0)
    for problem in problems:
        if verbose:
            print("\nproblem", problem)

        f1_train = 0
        f1_val = 0
        f1_test = 0

        # Get raw datasets
        X_train = dictSoal[problem].RESPONSE
        Y_train = dictSoal[problem].LABEL.to_numpy()
        X_val = dictSoal[problem].RESPONSE
        Y_val = dictSoal[problem].LABEL.to_numpy()
        X_test = dictTest[problem].RESPONSE
        Y_test = dictTest[problem].LABEL.to_numpy()

        # Get embeddings
        embed_model = embeddingModel
        if (params.embedding == "multi"):
          embed_model = embeddingModel2
        X_test_embed = np.c_[vectorize(X_test, embed_model), findLen(X_test)]
        X_train_embed = np.c_[vectorize(X_train, embed_model), findLen(X_train)]
        X_val_embed = np.c_[vectorize(X_val, embed_model), findLen(X_val)]

        # Convert to DataLoaders
        trainDataset = AesDataset(X_train_embed, Y_train)
        trainLoader = DataLoader(trainDataset, batch_size=params.init_bs)

        validDataset = AesDataset(X_val_embed, Y_val)
        validLoader = DataLoader(validDataset)

        testDataset = AesDataset(X_test_embed, Y_test)
        testLoader = DataLoader(testDataset)

        # LSP is giving this errors for some reason. Even though it runs fine
        if params.smote:
            if problem=="A":
                sampler = SMOTE(sampling_strategy=0.5, random_state=113)
                X_train_embed, Y_train = sampler.fit_resample(X_train_embed, Y_train)
            else:
                sampler = SMOTE(sampling_strategy=0.9, random_state=113)
                X_train_embed, Y_train = sampler.fit_resample(X_train_embed, Y_train)

        model = NeuralNetwork().to(device)
        if (params.model == "nn2"):
          model = NeuralNetwork2().to(device)

        train(trainDataset, model, params)

        f1_val, val_errs = evaluate_model(model, X_val, Y_val, validLoader)
        f1_test, test_errs = evaluate_model(model, X_test, Y_test, testLoader)
        if (verbose):
          print('val', f1_val)
          print('test', f1_test)

        pd.DataFrame(test_errs).to_csv('errors.csv')


        avg_f1_val += f1_val
        avg_f1_test += f1_test

        if verbose:
            print("Average F1 val", problem, f1_val/len(problems))
            print("Average F1 test", problem, f1_test/len(problems))

    return avg_f1_val, avg_f1_test

##### Run with cv

In [ ]:
def run_cv(params: ModelParams,  problems=["A"], fixed_seed=True, verbose=False):
    skf = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
    avg_f1_val = 0
    avg_f1_test = 0
    if(fixed_seed):
      set_seed(0)

    for problem in problems:
        if verbose:
            print("\nproblem", problem)
        for i, (train_index, test_index) in enumerate(skf.split(dictSoal[problem].RESPONSE, dictSoal[problem].LABEL)):
            if verbose:
                print("---- FOLD",i)

            # Get raw datasets
            X_train = dictSoal[problem].RESPONSE.iloc[train_index]
            Y_train = dictSoal[problem].LABEL.iloc[train_index].to_numpy()
            X_val = dictSoal[problem].RESPONSE.iloc[test_index]
            Y_val = dictSoal[problem].LABEL.iloc[test_index].to_numpy()
            X_test = dictTest[problem].RESPONSE
            Y_test = dictTest[problem].LABEL.to_numpy()

            # Get embeddings
            embed_model = embeddingModel
            if (params.embedding == "multi"):
              embed_model = embeddingModel2
            X_test = np.c_[vectorize(X_test, embed_model), findLen(X_test)]
            X_train = np.c_[vectorize(X_train, embed_model), findLen(X_train)]
            X_val = np.c_[vectorize(X_val, embed_model), findLen(X_val)]

            # Convert to DataLoaders
            trainDataset = AesDataset(X_train,Y_train)
            trainLoader = DataLoader(trainDataset, batch_size=params.init_bs)

            validDataset = AesDataset(X_val, Y_val)
            validLoader = DataLoader(validDataset, batch_size=params.init_bs)

            testDataset = AesDataset(X_test, Y_test)
            testLoader = DataLoader(testDataset)

            # LSP is giving this errors for some reason. Even though it runs fine
            if params.smote:
                if problem=="A":
                    sampler = SMOTE(sampling_strategy=0.5, random_state=113)
                    X_train, Y_train = sampler.fit_resample(X_train, Y_train)
                else:
                    sampler = SMOTE(sampling_strategy=0.9, random_state=113)
                    X_train, Y_train = sampler.fit_resample(X_train, Y_train)


            model = NeuralNetwork().to(device)
            if (params.model == "nn2"):
              model = NeuralNetwork2().to(device)
            train(trainDataset, model, params)

            f1_val, val_errs = evaluate_model(model, X_val, Y_val, validLoader)
            f1_test, test_errs = evaluate_model(model, X_test, Y_test, testLoader)
            if (verbose):
              print('val', f1_val)
              print('test', f1_test)

            avg_f1_val += f1_val
            avg_f1_test += f1_test

        avg_f1_val /= 5
        avg_f1_test /= 5

        if verbose:
            print("Average F1 val", problem, avg_f1_val)
            print("Average F1 test", problem, avg_f1_test)

    return avg_f1_val, avg_f1_test

# Test Samples

In [ ]:
def run_samples(params: ModelParams = ModelParams(), task="A-bt", samples=30):
  avg_val = 0
  avg_test = 0
  max_test = 0

  for i in range(samples):
    print(i)
    val, test = run(params, [task], fixed_seed=False, verbose=False)
    avg_val += val
    avg_test += test
    max_test = test if test > max_test else max_test

  avg_val = avg_val / samples
  avg_test = avg_test / samples
  print()
  print('avg_val', avg_val)
  print('avg_test', avg_test)
  print('max test', max_test)

In [ ]:
params =  ModelParams()

# run_samples(params, "B")
# run_samples(params, "B-bt")
# run_samples(params, "B-p3")
# run_samples(params, "B-p3-bt")

# run_samples(params, "A")
# run_samples(params, "A-bt")
# run_samples(params, "A-p3")
# run_samples(params, "A-p3-bt")


# Hyperparameter Tuning

In [ ]:
import optuna
from optuna.samplers import TPESampler

path_out = "/Models/"

TASK = "A-p3-bt"

set_seed(0)

def objective(trial):
    epochs = trial.suggest_int('epochs', 4, 10, 2)
    init_bs = trial.suggest_int('init_bs', 2, 8, 2)
    bs_inc = trial.suggest_int('bs_inc', 0, 4, 1)
    inc_freq = trial.suggest_int('inc_freq', 2, 4, 1)
    smote = trial.suggest_categorical('smote', [True, False])
    model = trial.suggest_categorical('model', ['nn1', 'nn2'])
    embedding = trial.suggest_categorical('embedding', ['multi', 'indo'])

    params =  ModelParams(
      epochs=epochs,
      init_bs=init_bs,
      bs_inc=bs_inc,
      inc_freq=inc_freq,
      smote=smote,
      model=model,
      embedding=embedding,
    )

    f1_val, f1_test = run_cv(params, [TASK], verbose=True)
    with open(f"{str(trial.number)}.pkl", "wb") as f:
        pickle.dump(study, f)

    return f1_val


sampler = TPESampler(seed=113)
study = optuna.create_study(direction='maximize', sampler=sampler)
study.optimize(objective, n_trials=200)


In [ ]:
with open("test.pkl", "wb") as f:
    pickle.dump(study, f)
print("======================================")
print(study.best_params)
print(study.best_value)

In [ ]:
params =  ModelParams(epochs=8, init_bs=4, bs_inc=0, inc_freq=2, smote=False, model='nn1', embedding='indo')
run_samples(params, "B-p3-bt")

In [ ]:
params =  ModelParams(epochs=8, init_bs=4, bs_inc=0, inc_freq=4, smote=False, model='nn1', embedding='indo')
run_samples(params, "A-p3-bt")